In [ ]:
import os
import sys

import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import numpy as np
try:
    import open3d as o3d
except ImportError:
    print(sys.exc_info())
import pandas as pd
from scipy import interpolate
import quadpy

from dosipy.utils.dataloader import load_ear_data
from dosipy.utils.integrate import elementwise_quad, elementwise_dblquad
from dosipy.utils.viz import (set_colorblind, scatter_2d, scatter_3d,
                              fig_config, colormap_from_array)
sys.path.append('..')
from helpers import (clean_df, export_pcd, export_fields,
                       poynting_vector, get_imcolors, export_rect_idx)

In [ ]:
# set input data

polarization = 'te'
f = 26

if f == 26:
    edge_length = 20
else:
    edge_length = 10
area = edge_length ** 2

In [ ]:
# load surface data

if polarization == 'te':
    origin = [-25.28, 17.25]
else:
    origin = [-19.77, 5.22]

df = load_ear_data(polarization, f, surface='front')
df = clean_df(df)
xyz = export_pcd(df)
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz)
center = pcd.get_center()
cframe = o3d.geometry.TriangleMesh.create_coordinate_frame(
    size=9, origin=center+np.array([6, -25, -20])
)
#o3d.visualization.draw_geometries([pcd, cframe])

In [ ]:
# extract points visible from the plane wave incidence POV (-x direction)

diameter = np.linalg.norm(
    pcd.get_max_bound() - pcd.get_min_bound()
)
radius = 10 ** 5
camera = [center[0] + diameter, center[1] * 2.5, center[2]]
_, mask = pcd.hidden_point_removal(camera, radius)
xyz_zy = xyz[mask]

In [ ]:
# compute averaged APD

avg_center = [origin[0]+edge_length/2,
              origin[1]+edge_length/2]
area = edge_length ** 2
pAPD_origin, idx_rect = export_rect_idx(xyz=xyz_zy,
                                        center=avg_center,
                                        edge_length=edge_length,
                                        view='zy')
xyz_rect = xyz_zy[idx_rect]

In [ ]:
pcd_rect = o3d.geometry.PointCloud()
pcd_rect.points = o3d.utility.Vector3dVector(xyz_rect)
pcd_rect.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamKNN(30))
pcd_rect.orient_normals_consistent_tangent_plane(30)
#o3d.visualization.draw_geometries([pcd_rect], point_show_normal=True)

# Constant value

In [ ]:
n = np.asarray(pcd_rect.normals)
APD_rect = np.ones((n.shape[0]))

In [ ]:
set_colorblind()
label = '$APD$ [W/m$^2$]'
fig_config(latex=True, scaler=1.5, text_size=16)
fig, ax = scatter_2d({'$y$ [mm]': xyz_rect[:, 1],
                      '$z$ [mm]': xyz_rect[:, 2],
                      label: APD_rect}, s=1)

In [ ]:
degree = 21
points = np.c_[xyz_rect[:, 2], xyz_rect[:, 1]]
bbox = [origin[0], origin[0]+edge_length, origin[1], origin[1]+edge_length]

In [ ]:
func = interpolate.SmoothBivariateSpline(points[:, 0], points[:, 1], APD_rect, bbox=bbox, kx=3, ky=3)

scheme = quadpy.c2.get_good_scheme(degree)
val = scheme.integrate(lambda x: func(x[0], x[1], grid=False),
                       quadpy.c2.rectangle_points([origin[0], origin[0]+edge_length],
                                                  [origin[1], origin[1]+edge_length]))
APD_av = val / area
APD_av

# Surface area

In [ ]:
n = np.asarray(pcd_rect.normals)
APD_rect = n[:, 0] + n[:, 1] + n[:, 2]

In [ ]:
set_colorblind()
label = '$APD$ [W/m$^2$]'
fig_config(latex=True, scaler=1.5, text_size=16)
fig, ax = scatter_2d({'$y$ [mm]': xyz_rect[:, 1],
                      '$z$ [mm]': xyz_rect[:, 2],
                      label: APD_rect}, s=1)

In [ ]:
degree = 21
points = np.c_[xyz_rect[:, 2], xyz_rect[:, 1]]
bbox = [origin[0], origin[0]+edge_length, origin[1], origin[1]+edge_length]

In [ ]:
func = interpolate.SmoothBivariateSpline(points[:, 0], points[:, 1], APD_rect, bbox=bbox, kx=3, ky=3)

scheme = quadpy.c2.get_good_scheme(degree)
val = scheme.integrate(lambda x: func(x[0], x[1], grid=False),
                       quadpy.c2.rectangle_points([origin[0], origin[0]+edge_length],
                                                  [origin[1], origin[1]+edge_length]))
val